In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn import svm
import time 
start=time.time()

In [ ]:
import urllib.request
urllib.request.urlretrieve ("https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz", "a.tar.gz")


In [ ]:
import tarfile
tar = tarfile.open("a.tar.gz")
tar.extractall()
tar.close()


## Loading Dataset from link

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pickle


def unpickle(file):
 '''Load byte data from file'''
 with open(file, 'rb') as f:
  data = pickle.load(f, encoding='latin-1')
  return data


def load_cifar10_data(data_dir):
 '''Return train_data, train_labels, test_data, test_labels
 The shape of data is 32 x 32 x3'''
 train_data = None
 train_labels = []

 for i in range(1, 6):
  data_dic = unpickle(data_dir + "/data_batch_{}".format(i))
  if i == 1:
   train_data = data_dic['data']
  else:
   train_data = np.vstack((train_data, data_dic['data']))
  train_labels += data_dic['labels']

 test_data_dic = unpickle(data_dir + "/test_batch")
 test_data = test_data_dic['data']
 test_labels = test_data_dic['labels']

 train_data = train_data.reshape((len(train_data), 3, 32, 32))
 train_data = np.rollaxis(train_data, 1, 4)
 train_labels = np.array(train_labels)

 test_data = test_data.reshape((len(test_data), 3, 32, 32))
 test_data = np.rollaxis(test_data, 1, 4)
 test_labels = np.array(test_labels)

 return train_data, train_labels, test_data, test_labels

data_dir = 'cifar-10-batches-py'

train_data, train_labels, test_data, test_labels = load_cifar10_data(data_dir)

print(train_data.shape)
print(train_labels.shape)

print(test_data.shape)
print(test_labels.shape)


## Scaling the Data

In [ ]:
x_train = train_data.reshape(train_data.shape[0],-1)
x_test = test_data.reshape(test_data.shape[0], -1)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train_scaled = sc.fit_transform(x_train)


In [ ]:
x_test_scaled = sc.transform(x_test)


In [ ]:
y_train = train_labels
y_test = test_labels

# Applying PCA

In [ ]:
pca = PCA()
pca.fit_transform(x_train_scaled)

In [ ]:
# Calculating optimal k to have 0.95 variance

k = 0
total = sum(pca.explained_variance_)
current_sum = 0

while(current_sum / total < 0.99):
    current_sum += pca.explained_variance_[k]
    k += 1
k

In [ ]:
pca_cifar = PCA(n_components=k,whiten=True)
x_train_pca_cifar = pca_cifar.fit_transform(x_train_scaled)
x_test_pca_cifar = pca_cifar.transform(x_test_scaled)

# Logisitic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
lr = LogisticRegression()
lr.fit(x_train_pca_cifar, y_train)

In [ ]:
y_pred_lr = lr.predict(x_test_pca_cifar)
logistic_regression_score = accuracy_score(y_test, y_pred_lr)
logistic_regression_score

# SVM

In [ ]:
svc = svm.SVC(C= 100, gamma=0.005)

In [ ]:
svc.fit(x_train_pca_cifar,y_train)

In [ ]:
y_pred_svm = svc.predict(x_test_pca_cifar)
svc_score = accuracy_score(y_test, y_pred_svm)
svc_score
print("time is :",(time.time()-start)/60," mins")

## Accuracy - 55.52%

We can 55.52% accuracy using pca and SVM
This can be improved by retaining more data than 95%